# Setup
* The following steps are used for setup only

In [ ]:
# Update below
bq_dataset = "DATASET_NAME"
bq_project = "PROJECT_NAME"
bq_location = "US"


In [ ]:
!pip install --upgrade firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
# cred = credentials.Certificate('path/to/serviceAccount.json')
try:
  app = firebase_admin.initialize_app()
except:
  pass

db = firestore.client(database_id="o11ydemo")


# Queries

## Looking for the logs that we generated and logged to stdout.  They are in JSON format that we can natively use with BigQuery.

In [ ]:
import bigframes.pandas as bpd

# Look for logs with LLM called, using the JSON format
sql = f"""
select TIMESTAMP(jsonPayload.timestamp) AS parsed_timestamp,*
from `{bq_project}.observability_demo.run_googleapis_com_stdout`
where jsonPayload.message = 'scoring question'
"""
log_df = bpd.read_gbq_query(sql)
log_df.head(10)

## Pulling the scoring information, which answers were answered correctly. We did not log the full question on purpose because we can show that we can now join it.

In [ ]:
sql = f"""
select jsonPayload.timestamp,jsonPayload.qid, jsonPayload.correct
from `{bq_project}.observability_demo.run_googleapis_com_stdout`
where jsonPayload.message = 'scoring question'
"""
print(sql)
log_df = bpd.read_gbq_query(sql)
log_df.head(50)


## We connect to Firestore and and pull down all the questions into a dataframe to be used by our notebook.

In [ ]:
log_dict = log_df.to_dict()

questions_array = []
for collection in ["FLASH","FLASHLITE","GEMMA3"]:
  doc_ref = db.collection(collection)
  docs = doc_ref.stream()
  for doc in docs:
      # print(f"{doc.id} => {doc.to_dict()}")
      questions_dict = doc.to_dict()
      questions_dict['qid'] = doc.id
      questions_array.append(questions_dict)

questions_df = bpd.DataFrame(questions_array)
second_questions_df = questions_df.copy()
questions_df.head(100)

## Merge the two together - essentially a join, you've now joined Firestore data with your logs, enriching them!

In [ ]:
lookup_df = log_df.merge(questions_df,left_on='qid',right_on='qid')

## We can do some preliminary counts to show how many, split by model, were correctly answered and plot it!

In [ ]:
count_df = lookup_df.groupby(["code","correct"]).count()
count_df.head(100)

In [ ]:

# Plot value counts
count_df.plot(kind='bar', color='skyblue')

## Finally, we pull your prompt to see the responses from the various models.

In [ ]:
latest_sql = """
with filter_table as (
    select jsonPayload.session_id as session_id, RANK() OVER (PARTITION BY jsonPayload.message ORDER BY TIMESTAMP(jsonPayload.timestamp) DESC) AS rnk
    from `observability_demo.run_googleapis_com_stdout`
    where jsonPayload.message = 'scoring question' and jsonPayload.session_id is not null and jsonPayload.answer is not null
    qualify rnk < 10
)
select jsonPayload.session_id, jsonPayload.timestamp, jsonPayload.qid, jsonPayload.correct, jsonPayload.model, jsonPayload.answer
from `observability_demo.run_googleapis_com_stdout`
where jsonPayload.message = 'scoring question'
and jsonPayload.session_id in (
  select session_id
  from filter_table
)
"""
latest_df = bpd.read_gbq_query(latest_sql)

latest_lookup_df = latest_df.merge(questions_df,left_on='qid',right_on='qid')
latest_lookup_df.head(1000)

In [ ]:
prompts_sql = """
with inner_query as (
select session_id
from (
  select JSON_QUERY(cast(data as string),'$.session_id') as session_id, JSON_QUERY(cast(data as string),'$.model'), JSON_QUERY(cast(data as string),'$.prompt') , JSON_QUERY(cast(data as string),'$.formatted_response') , cast(REPLACE(JSON_QUERY(cast(data as string),'$.session_id'),'"','') as int),
  rank() over (ORDER BY cast(REPLACE(JSON_QUERY(cast(data as string),'$.session_id'),'"','') as int) desc) as rnk
  from observability_demo.logPromptsAndResponses
  qualify rnk < 10)
)
select JSON_QUERY(cast(data as string),'$.session_id'), JSON_QUERY(cast(data as string),'$.model'), JSON_QUERY(cast(data as string),'$.prompt') , JSON_QUERY(cast(data as string),'$.formatted_response') , cast(REPLACE(JSON_QUERY(cast(data as string),'$.session_id'),'"','') as int)
from observability_demo.logPromptsAndResponses
where JSON_QUERY(cast(data as string),'$.session_id') in (
  select *
  from inner_query
)
"""
prompts_df = bpd.read_gbq_query(prompts_sql)

prompts_df.head(100)